In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline


# 1. CSV to ARFF

In [ ]:
#Since we are using Python, we do  not need to complete this step

# Loading Data

## pandas.read_csv

In [ ]:
file_path = "../../data/raw/"

X = pd.read_csv(f"{file_path}x_train_gr_smpl.csv", delimiter=',')
X.info()

In [ ]:
# image = X.loc[[600]].values
# image = image[0].reshape((48,48))
# image.shape
# plt.imshow(image)

In [ ]:
Y = pd.read_csv(f"{file_path}y_train_smpl.csv", delimiter=',')
Y.columns = ['target']

y0 = pd.read_csv(f"{file_path}y_train_smpl_0.csv", delimiter=',')
y0.columns = ['target']

y1 = pd.read_csv(f"{file_path}y_train_smpl_1.csv", delimiter=',')
y1.columns = ['target']

y2 = pd.read_csv(f"{file_path}y_train_smpl_2.csv", delimiter=',')
y2.columns = ['target']

y3 = pd.read_csv(f"{file_path}y_train_smpl_3.csv", delimiter=',')
y3.columns = ['target']

y4 = pd.read_csv(f"{file_path}y_train_smpl_4.csv", delimiter=',')
y4.columns = ['target']

y5 = pd.read_csv(f"{file_path}y_train_smpl_5.csv", delimiter=',')
y5.columns = ['target']

y6 = pd.read_csv(f"{file_path}y_train_smpl_6.csv", delimiter=',')
y6.columns = ['target']

y7 = pd.read_csv(f"{file_path}y_train_smpl_7.csv", delimiter=',')
y7.columns = ['target']

y8 = pd.read_csv(f"{file_path}y_train_smpl_8.csv", delimiter=',')
y8.columns = ['target']

y9 = pd.read_csv(f"{file_path}y_train_smpl_9.csv", delimiter=',')
y9.columns = ['target']

Y.tail()

# Create datasets

In [ ]:
train_smpl_0 = pd.concat([X, y0], axis=1)
train_smpl_1 = pd.concat([X, y1], axis=1)

In [ ]:
train_smpl_2 = pd.concat([X, y2], axis=1)
train_smpl_3 = pd.concat([X, y3], axis=1)

In [ ]:
train_smpl_4 = pd.concat([X, y4], axis=1)
train_smpl_5 = pd.concat([X, y5], axis=1)

In [ ]:
train_smpl_6 = pd.concat([X, y6], axis=1)
train_smpl_7 = pd.concat([X, y7], axis=1)

In [ ]:
train_smpl_8 = pd.concat([X, y8], axis=1)
train_smpl_9 = pd.concat([X, y9], axis=1)

In [ ]:
train_smpl = pd.concat([X, Y], axis=1)
train_smpl.head()

In [ ]:
# train_smpl_0.head()

# 2. Data Randomisation

## sklearn.utils.suffle

In [ ]:
train_smpl = shuffle(train_smpl, random_state=42)
train_smpl_0 = shuffle(train_smpl_0, random_state=42)
train_smpl_1 = shuffle(train_smpl_1, random_state=42)
train_smpl_2 = shuffle(train_smpl_2, random_state=42)
train_smpl_3 = shuffle(train_smpl_3, random_state=42)
train_smpl_4 = shuffle(train_smpl_4, random_state=42)
train_smpl_5 = shuffle(train_smpl_5, random_state=42)
train_smpl_6 = shuffle(train_smpl_6, random_state=42)
train_smpl_7 = shuffle(train_smpl_7, random_state=42)
train_smpl_8 = shuffle(train_smpl_8, random_state=42)
train_smpl_9 = shuffle(train_smpl_9, random_state=42)
train_smpl.head()

# class distribution

In [ ]:
#train_smpl.hist(column='target')
plt.hist(Y['target'], bins='auto')

In [ ]:
train_smpl.target.value_counts()

# 3. Reducing the size

#### Because the data runs the data as a Python file, we do not need to reduce the size of our data set. 



In [ ]:
train_smpl.info()

In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.pipeline import make_pipeline

In [ ]:
# ros = RandomOverSampler(random_state=0)
# X_oversampled, y_oversampled = ros.fit_resample(X, Y['target'])
# plt.hist(y_oversampled, bins='auto')

In [ ]:

oversample_ratio={2: 1000,  6: 1000, 7: 1000, 9: 1000}
undersample_ratio={0: 1000, 1: 1000,  3: 1000, 4: 1000, 5: 1000, 8: 1000}
pipe = make_pipeline(SMOTE(sampling_strategy=oversample_ratio, n_jobs=7), NearMiss(sampling_strategy=undersample_ratio, n_jobs=7))

X_resampled, y_resampled = pipe.fit_resample(X, Y['target'])
plt.hist(y_resampled, bins='auto')

In [ ]:

# rus = RandomUnderSampler(random_state=0)
# X_undersampled, y_undersampled = rus.fit_sample(X, Y)
# plt.hist(y_undersampled, bins='auto')

In [ ]:
# X_oversampled = pd.DataFrame(data=X_oversampled)
# y_oversampled = pd.DataFrame(data=y_oversampled)
# y_oversampled.columns=(['target'])
# train_oversampled = pd.concat([X_oversampled, y_oversampled], axis=1)
# train_oversampled = shuffle(train_oversampled, random_state=42)
# train_oversampled.info()

In [ ]:
X_resampled = pd.DataFrame(data=X_resampled)
y_resampled = pd.DataFrame(data=y_resampled)
y_resampled.columns=(['target'])
train_resampled = pd.concat([X_resampled, y_resampled], axis=1)
train_resampled = shuffle(train_resampled, random_state=42)
train_resampled.info()

In [ ]:
# X_undersampled = pd.DataFrame(data=X_undersampled)
# y_undersampled = pd.DataFrame(data=y_undersampled)
# y_undersampled.columns=(['target'])
# train_undersampled = pd.concat([X_undersampled, y_undersampled], axis=1)
# train_undersampled = shuffle(train_undersampled, random_state=42)
# train_undersampled.info()

# 4 - 6: NB, Features/Attributes Selection, & Improving #5's Classification  

## correlation matrix

In [ ]:

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
print(f"train_smpl shape:{train_smpl.shape}")

best_20 = SelectKBest(chi2, k=20).fit_transform(train_resampled[train_resampled.columns[:2303]], train_resampled['target'])
best_20 = pd.DataFrame(best_20)
train_smpl_20 = pd.concat([best_20, train_resampled['target']], axis=1)
print(f"train_smpl_20 shape:{train_smpl_20.shape}")

best_50 = SelectKBest(chi2, k=50).fit_transform(train_resampled[train_resampled.columns[:2303]], train_resampled['target'])
best_50 = pd.DataFrame(best_50)
train_smpl_50 = pd.concat([best_50, train_resampled['target']], axis=1)
print(f"train_smpl_50 shape:{train_smpl_50.shape}")

best_100 = SelectKBest(chi2, k=100).fit_transform(train_resampled[train_resampled.columns[:2303]], train_resampled['target'])
best_100 = pd.DataFrame(best_100)
train_smpl_100 = pd.concat([best_100, train_resampled['target']], axis=1)
print(f"train_smpl_100 shape:{train_smpl_100.shape}")


# Test Train Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_smpl[train_smpl.columns[:2303]], train_smpl['target'], test_size=0.33, random_state=42)
# X_train_ovr, X_test_ovr, y_train_ovr, y_test_ovr = train_test_split(train_oversampled[train_oversampled.columns[:2303]], train_oversampled['target'], test_size=0.33, random_state=42)
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split(train_resampled[train_resampled.columns[:2303]], train_resampled['target'], test_size=0.33, random_state=42)
# X_train_udr, X_test_udr, y_train_udr, y_test_udr = train_test_split(train_undersampled[train_undersampled.columns[:2303]], train_undersampled['target'], test_size=0.33, random_state=42)

X_train_20, X_test_20, y_train_20, y_test_20 = train_test_split(train_smpl_20[train_smpl_20.columns[:20]], train_smpl_20['target'], test_size=0.33, random_state=42)
X_train_50, X_test_50, y_train_50, y_test_50 = train_test_split(train_smpl_50[train_smpl_50.columns[:50]], train_smpl_50['target'], test_size=0.33, random_state=42)
X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(train_smpl_100[train_smpl_100.columns[:100]], train_smpl_100['target'], test_size=0.33, random_state=42)


X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(train_smpl_0[train_smpl_0.columns[:2303]], train_smpl_0['target'], test_size=0.33, random_state=42)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(train_smpl_1[train_smpl_1.columns[:2303]], train_smpl_1['target'], test_size=0.33, random_state=42)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(train_smpl_2[train_smpl_2.columns[:2303]], train_smpl_2['target'], test_size=0.33, random_state=42)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(train_smpl_3[train_smpl_3.columns[:2303]], train_smpl_3['target'], test_size=0.33, random_state=42)
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(train_smpl_4[train_smpl_4.columns[:2303]], train_smpl_4['target'], test_size=0.33, random_state=42)
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(train_smpl_5[train_smpl_5.columns[:2303]], train_smpl_5['target'], test_size=0.33, random_state=42)
X_train_6, X_test_6, y_train_6, y_test_6 = train_test_split(train_smpl_6[train_smpl_6.columns[:2303]], train_smpl_6['target'], test_size=0.33, random_state=42)
X_train_7, X_test_7, y_train_7, y_test_7 = train_test_split(train_smpl_7[train_smpl_7.columns[:2303]], train_smpl_7['target'], test_size=0.33, random_state=42)
X_train_8, X_test_8, y_train_8, y_test_8 = train_test_split(train_smpl_8[train_smpl_8.columns[:2303]], train_smpl_8['target'], test_size=0.33, random_state=42)
X_train_9, X_test_9, y_train_9, y_test_9 = train_test_split(train_smpl_9[train_smpl_9.columns[:2303]], train_smpl_9['target'], test_size=0.33, random_state=42)

X_train_10classes = [X_train_0, X_train_1, X_train_2, X_train_3, X_train_4, X_train_5, X_train_6, X_train_7, X_train_8, X_train_9]
X_test_10classes = [X_test_0, X_test_1, X_test_2, X_test_3, X_test_4, X_test_5, X_test_6, X_test_7, X_test_8, X_test_9]
y_train_10classes = [y_train_0, y_train_1, y_train_2, y_train_3, y_train_4, y_train_5, y_train_6, y_train_7, y_train_8, y_train_9]
y_test_10classes = [y_test_0, y_test_1, y_test_2, y_test_3, y_test_4, y_test_5, y_test_6, y_test_7, y_test_8, y_test_9]

# Modeling

## Multinomial Naive Bayes model (multi-class classifier)

### Before Features/Attributes Selection

In [ ]:
# In regards to problem 4: Because we are not using Weka, we did not need to apply any
# filters to the data before running Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
# from sklearn.naive_bayes import MultinomialNB
# clf = MultinomialNB()
# clf.fit(X_train_ovr, y_train_ovr)
# clf.score(X_test_ovr, y_test_ovr)

In [ ]:
clf = MultinomialNB()
clf.fit(X_train_res, y_train_res)
clf.score(X_test_res, y_test_res)

In [ ]:
# from sklearn.naive_bayes import MultinomialNB
# clf = MultinomialNB()
# clf.fit(X_train_udr, y_train_udr)
# clf.score(X_test_udr, y_test_udr)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = clf.predict(X_test_res)
conf_mat = confusion_matrix(y_test_res, y_pred)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
        https://stackoverflow.com/a/50386871
    """

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
plot_confusion_matrix(conf_mat, target_names=y_test.unique())

In [ ]:
# looking for devious class labels that take high numbers of misclassifications
for i in range(conf_mat.shape[1]):
    column = conf_mat.T[i]
    misclassifications = column.sum() - column[i]
    print(i,misclassifications)

### 5. For each of the 10 train_smpl_label files, record the first 10 fields, in order of the absolute correlation value for each street sign.

In [ ]:
from scipy import stats
fileList = [train_smpl_0, train_smpl_1,train_smpl_2, train_smpl_3, train_smpl_4,
            train_smpl_5, train_smpl_6, train_smpl_7, train_smpl_8, train_smpl_9]
for f, file in enumerate(fileList):
    corrArr = []
    for j in file.columns[:-1]:
        corrVal, pVal = stats.pearsonr(file.iloc[:,int(j)], file.iloc[:, -1])
        corrArr.append(corrVal**2)
    print(f"File {f}:")
    print(sorted([(x,i) for (i,x) in enumerate(corrArr)], reverse=True)[:10])

### After Features/Attributes Selection (20/50/100)

In [ ]:
from sklearn.naive_bayes import GaussianNB

X_trains_reduced = [  X_train_20, X_train_50, X_train_100]
X_tests_reduced = [ X_test_20, X_test_50, X_test_100]
y_trains_reduced = [ y_train_20, y_train_50, y_train_100]
y_tests_reduced = [ y_test_20, y_test_50, y_test_100]

# Gaussian Model on Multiclass dataset ?
for X_trains, X_tests, y_trains, y_tests in zip(X_trains_reduced, X_tests_reduced, y_trains_reduced, y_tests_reduced):    
    clf = GaussianNB()
    clf.fit(X_trains, y_trains)
    print(f"Gaussian NB {X_trains.shape[1]}: {clf.score(X_tests, y_tests)}")

for X_trains, X_tests, y_trains, y_tests in zip(X_trains_reduced, X_tests_reduced, y_trains_reduced, y_tests_reduced):    
    clf = MultinomialNB()
    clf.fit(X_trains, y_trains)
    print(f"Multinomial NB {X_trains.shape[1]}: {clf.score(X_tests, y_tests)}")


## Gaussian Naive Bayes model (mono-class classifier)

In [ ]:
scores = []
for X_train, X_test, y_train, y_test in zip(X_train_10classes, X_test_10classes, y_train_10classes, y_test_10classes):    
    clf = GaussianNB()
    clf.fit(X_train, y_train)
    scores.append(clf.score(X_test, y_test))
    print(f"Gaussian NB 10 classes:{clf.score(X_test, y_test)}")
print("Gaussian NB Mean: ", sum(scores)/len(scores))

scores2 = []
for X_train, X_test, y_train, y_test in zip(X_train_10classes, X_test_10classes, y_train_10classes, y_test_10classes):    
    clf = MultinomialNB()
    clf.fit(X_train, y_train)
    scores2.append(clf.score(X_test, y_test))
    print(f"Multinomial NB 10 classes: {clf.score(X_test, y_test)}")
print("Multinomial NB Mean: ", sum(scores2)/len(scores2))


In [ ]:
#Multinomial Nb is more accurate than Gaussian Nb ( 0.75 > 0.69 ) on binary datasets

# Analysis for 4 & 7

### 4. Explain the reason for choosing and using these filters. Once you can run the algorithm, record, compare and analyse the classifier’s accuracy on different classes (as given by the Weka Summary and the confusion matrix).

#### Since we are not using Weka, we did not need to apply any filters to the data before running Naive Bayes.

### 7. What kind of information about this data set did you learn, as a result of the above experiments? You should ask questions such as: Which streets signs are harder to recognise? Which street signs are most easily confused? Which attributes (fields) are more reliable and which are less reliable in classification of street signs? What was the purpose of Tasks 5 and 6? What would happen if the data sets you used in Tasks 4, 5 and 6 were not randomised? What would happen if there is cross-correlation between the non-class attributes? You will get more marks for more interesting and ``out of the box” questions and answers. Explain your conclusions logically and formally, using the material from the lecture notes and from your own reading to interpret the results that Weka produces.

#### Analysis....

## Conclusions from initial experiments (q7)

Using the multilabel confusion matrix, it is apparent that the multinomial Naïve Bayes classifier struggled with some classes, predicting them incorrectly most of the time. 

In particular, class 8 was more often classified as class 1 (probability 0.278), class 7 (probability 0.2023) and class 6 (probability 0.1974) before itself (probability 0.1283).

//It looks like the classifier was confused by ...



The strongest misclassification in the matrix is true members of class 3 being classified as class 7 (probability 0.2023). However, there is relatively very little of the reverse misclassification (true class 7 predicted as class 3).

//^^ investigate this?



Class 7 punches above its weight as the most abundant misclassification which is perhaps due to the relative infrequency of data for class 7. There must be some other factor, as there are less populated classes.

